In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 6.4MB/s 
     |████████████████████████████████| 901kB 16.1MB/s 
     |████████████████████████████████| 3.3MB 34.4MB/s 


In [ ]:
from transformers import TFMobileBertModel
import pandas as pd
from google.colab import drive
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
import json
drive.mount('/content/drive')
import numpy as np

Mounted at /content/drive


In [ ]:
import os
import pprint # for pretty printing our device stats

if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print ('TPU address is', tpu_address)

    with tf.compat.v1.Session(tpu_address) as session:
      devices = session.list_devices()

    print('TPU devices:')
    pprint.pprint(devices)

TPU address is grpc://10.85.203.106:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, -3345723416498840148),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 8589934592, 8967475659710126863),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, -5370334693629456259),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 6608071019069748283),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8368606246539250738),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, -1372055816276813646),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 13983068925143006),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 6985691001245723491),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, -

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.85.203.106:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.85.203.106:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
class distilBertModels:
  def __init__(self,path):
    df = pd.read_csv(path)
    self.max_seq_len = 35
    bert_name = 'google/mobilebert-uncased'
    # self.tokenizer = AlbertTokenizer.from_pretrained(bert_name,add_special_tokens=True,max_length =self.max_seq_len,pad_to_max_length = True)
    self.train_set,self.test_set = self.prepare_data(df)
    with strategy.scope():
      self.bert = TFMobileBertModel.from_pretrained(bert_name)
      self.bert.trainable = False

  def prepare_data(self,df):
    df['encoded'] = df['encoded'].apply(lambda x: json.loads(x))
    df_tr = df[df['kfold']!=0].reset_index(drop=True)
    df_ts = df[df['kfold']==0].reset_index(drop=True)
    enc_tr = list(df_tr['encoded'])
    enc_ts = list(df_ts['encoded'])
    inp_ids_tr = [np.array(k[0]) for k in enc_tr]
    att_mks_tr = [np.array(k[1]) for k in enc_tr]
    inp_ids_ts = [np.array(k[0]) for k in enc_ts]
    att_mks_ts = [np.array(k[1]) for k in enc_ts]
    x_tr_inp_ids = tf.cast(tf.squeeze(tf.stack(inp_ids_tr)),dtype=tf.int32)
    x_tr_att_mks = tf.cast(tf.squeeze(tf.stack(att_mks_tr)),dtype=tf.int32)
    x_ts_inp_ids = tf.cast(tf.squeeze(tf.stack(inp_ids_ts)),dtype=tf.int32)
    x_ts_att_mks = tf.cast(tf.squeeze(tf.stack(att_mks_ts)),dtype=tf.int32)

    encoder = LabelEncoder()
    y_tr = np_utils.to_categorical(encoder.fit_transform(df_tr.label.values))
    y_ts = np_utils.to_categorical(encoder.fit_transform(df_ts.label.values))

    x_tr = [x_tr_inp_ids,x_tr_att_mks]
    x_ts = [x_ts_inp_ids,x_ts_att_mks]

    return ((x_tr,y_tr),(x_ts,y_ts))

  # def encode_data(self,x):
  #   encoded = self.tokenizer.encode_plus(x,add_special_tokens=True,max_length =self.max_seq_len,padding = 'max_length',truncation=True,return_attention_mask=True,return_tensors='tf')
  #   return encoded
  
  def process(self,ls):
  # for name in ls:
#       his = self.compile_model(name)
#       dfs = []
#       for i in range(len(his)):
#         df = pd.DataFrame.from_dict(his[i].history)
#         df['fold'] = i+1
#         df['epoch'] = [j+1 for j in range(len(df))]
#         dfs.append(df)
#       df_fin = pd.concat(dfs,ignore_index=True)  
#       df_fin.to_csv('/content/drive/MyDrive/Bert/Folds/sent_'+name+'_folds.csv',index=False,header=True)
    for name in ls:
      his = self.compile_model(name)
      df = pd.DataFrame.from_dict(his.history)
      df['epoch'] = [i+1 for i in range(len(df))]
      df.to_csv('/content/drive/MyDrive/MOBILE_BERT/Selected/sent_'+name+'.csv',index=False,header=True)
      print("\n------------------ PROCESS FOR MODEL: "+name+' COMPLETED ------------------\n')


  def compile_model(self,name):
    with strategy.scope():
      model = self.create_model(name)
      model.compile(loss='categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])
    
    
    print('\n------------------ '+name+' ------------------\n')
    model.summary()
    # his = []
    # for i in range(len(self.train_set_folds)):
#       print('\n**************** FOLD '+str(i+1)+' ****************')
    his = model.fit(x = self.train_set[0],y = self.train_set[1],validation_data=(self.test_set[0],self.test_set[1]),epochs=100,callbacks=[EarlyStopping()])
    print('\n**************** TRAINING OF MODEL: '+name+' COMPLETED****************\n')
    return his
    
  def create_model(self,name):
    inputs = tf.keras.layers.Input((self.max_seq_len,),dtype = tf.int32,name="input_ids")
    mask = tf.keras.layers.Input((self.max_seq_len,),dtype = tf.int32,name="attention_mask")

    pred = self.bert([inputs,mask])[0]

    if name == 'MOBILE-BERT-CONV1D': ## 1
      x = tf.keras.layers.Conv1D(32,3,activation='relu',padding='same')(pred)
      x = tf.keras.layers.AveragePooling1D()(x)
      x = tf.keras.layers.Conv1D(32,3,activation='relu',padding='same')(x)
      x = tf.keras.layers.GlobalAveragePooling1D()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(20,activation='relu')(x)
      x = tf.keras.layers.Dense(5,activation='softmax')(x)

    
    if name == 'MOBILE-BERT-CONV1D-BiGRU': ## 2
      x = tf.keras.layers.Conv1D(32,3,activation='relu',padding='same')(pred)
      x = tf.keras.layers.AveragePooling1D()(x)
      x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(512,activation='relu')(x)
      x = tf.keras.layers.Dense(128,activation='relu')(x)
      x = tf.keras.layers.Dense(32,activation='relu')(x)
      x = tf.keras.layers.Dense(5,activation='softmax')(x)
    
    if name == 'MOBILE-BERT-CONV1D-BiLSTM': ## 3
      x = tf.keras.layers.Conv1D(32,3,activation='relu',padding='same')(pred)
      x = tf.keras.layers.AveragePooling1D()(x)
      x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(512,activation='relu')(x)
      x = tf.keras.layers.Dense(128,activation='relu')(x)
      x = tf.keras.layers.Dense(32,activation='relu')(x)
      x = tf.keras.layers.Dense(5,activation='softmax')(x)    
      
    if name == 'MOBILE-BERT-CONV1D-BiRNN': ## 4
      x = tf.keras.layers.Conv1D(32,3,activation='relu',padding='same')(pred)
      x = tf.keras.layers.AveragePooling1D()(x)
      x = tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(512,activation='relu')(x)
      x = tf.keras.layers.Dense(128,activation='relu')(x)
      x = tf.keras.layers.Dense(32,activation='relu')(x)
      x = tf.keras.layers.Dense(5,activation='softmax')(x)

    if name == 'MOBILE-BERT-SEP-CONV1D': #5
      x = tf.keras.layers.SeparableConv1D(32,3,activation='relu',padding='same')(pred)
      x = tf.keras.layers.AveragePooling1D()(x)
      x = tf.keras.layers.SeparableConv1D(32,3,activation='relu',padding='same')(x)
      x = tf.keras.layers.AveragePooling1D()(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(256,activation='relu')(x)
      x = tf.keras.layers.Dense(64,activation='relu')(x)
      x = tf.keras.layers.Dense(16,activation='relu')(x)
      x = tf.keras.layers.Dense(5,activation='softmax')(x)
      

    if name == 'MOBILE-BERT-SEP-CONV1D-BiGRU': ## 6
      x = tf.keras.layers.SeparableConv1D(32,3,activation='relu',padding='same')(pred)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(x)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(256,activation='relu')(x)
      x = tf.keras.layers.Dense(64,activation='relu')(x)
      x = tf.keras.layers.Dense(16,activation='relu')(x)
      x = tf.keras.layers.Dense(5,activation='softmax')(x)
      
    if name == 'MOBILE-BERT-SEP-CONV1D-BiRNN': ## 7
      x = tf.keras.layers.SeparableConv1D(32,3,activation='relu',padding='same')(pred)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(x)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(256,activation='relu')(x)
      x = tf.keras.layers.Dense(64,activation='relu')(x)
      x = tf.keras.layers.Dense(16,activation='relu')(x)
      x = tf.keras.layers.Dense(5,activation='softmax')(x)
      
    if name == 'MOBILE-BERT-SEP-CONV1D-BiLSTM': ## 8
      x = tf.keras.layers.SeparableConv1D(32,3,activation='relu',padding='same')(pred)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(x)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(256,activation='relu')(x)
      x = tf.keras.layers.Dense(64,activation='relu')(x)
      x = tf.keras.layers.Dense(16,activation='relu')(x)
      x = tf.keras.layers.Dense(5,activation='softmax')(x)
      
      
    if name == 'MOBILE-BERT-BiGRU': ## 9
      x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(pred)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(256,activation='relu')(x)
      x = tf.keras.layers.Dense(64,activation='relu')(x)
      x = tf.keras.layers.Dense(16,activation='relu')(x)
      x = tf.keras.layers.Dense(5,activation='softmax')(x)
      
    if name == 'MOBILE-BERT-BiLSTM': ## 10
      x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(pred)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(256,activation='relu')(x)
      x = tf.keras.layers.Dense(64,activation='relu')(x)
      x = tf.keras.layers.Dense(16,activation='relu')(x)
      x = tf.keras.layers.Dense(5,activation='softmax')(x)
    
    if name == 'MOBILE-BERT-BiRNN': ## 11
      x = tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(pred)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(256,activation='relu')(x)
      x = tf.keras.layers.Dense(64,activation='relu')(x)
      x = tf.keras.layers.Dense(16,activation='relu')(x)
      x = tf.keras.layers.Dense(5,activation='softmax')(x)
    
      
    return tf.keras.Model([inputs,mask],x)




    

  

In [ ]:
bert = distilBertModels('/content/drive/MyDrive/sentiment data/tr_sen_fin_mob_bert_trans_inps.csv')

Some layers from the model checkpoint at google/mobilebert-uncased were not used when initializing TFMobileBertModel: ['predictions___cls', 'seq_relationship___cls']
- This IS expected if you are initializing TFMobileBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFMobileBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFMobileBertModel were initialized from the model checkpoint at google/mobilebert-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMobileBertModel for predictions without further training.


In [ ]:
bert.process(['MOBILE-BERT-SEP-CONV1D-BiRNN','MOBILE-BERT-SEP-CONV1D-BiLSTM','MOBILE-BERT-BiGRU','MOBILE-BERT-BiLSTM','MOBILE-BERT-BiRNN'])
# {1:'MOBILE-BERT-CONV1D',2:'MOBILE-BERT-CONV1D-BiGRU',3:'MOBILE-BERT-CONV1D-BiLSTM',4:'MOBILE-BERT-CONV1D-BiRNN',5:'MOBILE-BERT-SEP-CONV1D',6:'MOBILE-BERT-SEP-CONV1D-BiGRU',7:'MOBILE-BERT-SEP-CONV1D-BiRNN',8:'MOBILE-BERT-SEP-CONV1D-BiLSTM',9:'MOBILE-BERT-BiGRU',10:'MOBILE-BERT-BiLSTM',11:'MOBILE-BERT-BiRNN'}

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

17250/17250 [==============================] - ETA: 0s - loss: 0.9789 - accuracy: 0.6054WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


17250/17250 [==============================] - 895s 49ms/step - loss: 0.9789 - accuracy: 0.6054 - val_loss: 0.7815 - val_accuracy: 0.7007
Epoch 2/100
17250/17250 [==============================] - 819s 47ms/step - loss: 0.8241 - accuracy: 0.6801 - val_loss: 0.7512 - val_accuracy: 0.7122
Epoch 3/100
17250/17250 [==============================] - 823s 48ms/step - loss: 0.8017 - accuracy: 0.6896 - val_loss: 0.7490 - val_accuracy: 0.7137
Epoch 4/100
17250/17250 [==============================] - 822s 48ms/step - loss: 0.7933 - accuracy: 0.6933 - val_loss: 0.7454 - val_accuracy: 0.7170
Epoch 5/100
17250/17250 [==============================] - 843s 49ms/step - loss: 0.7861 - accuracy: 0.6979 - val_loss: 0.7312 - val_accuracy: 0.7215
Epoch 6/100
17250/17250 [==============================] - 787s 46ms/step - loss: 0.7798 - accuracy: 0.6996 - val_loss: 0.7274 - val_accuracy: 0.7249
Epoch 7/100
17250/17250 [==============================] - 807s 47ms/step - loss: 0.7769 - accuracy: 0.7009 - va


------------------ MOBILE-BERT-BiRNN ------------------

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 35)]         0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 35)]         0                                            
__________________________________________________________________________________________________
tf_mobile_bert_model (TFMobileB TFBaseModelOutputWit 24581888    input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
bidirectional_4 (Bidirectional) (N

17249/17250 [============================>.] - ETA: 0s - loss: 1.0494 - accuracy: 0.5698WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


17250/17250 [==============================] - 865s 47ms/step - loss: 1.0494 - accuracy: 0.5698 - val_loss: 0.9155 - val_accuracy: 0.6341
Epoch 2/100
17250/17250 [==============================] - 768s 45ms/step - loss: 0.9083 - accuracy: 0.6377 - val_loss: 0.8623 - val_accuracy: 0.6601
Epoch 3/100
17250/17250 [==============================] - 804s 47ms/step - loss: 0.8791 - accuracy: 0.6522 - val_loss: 0.8139 - val_accuracy: 0.6801
Epoch 4/100
17250/17250 [==============================] - 794s 46ms/step - loss: 0.8608 - accuracy: 0.6591 - val_loss: 0.8383 - val_accuracy: 0.6724

**************** TRAINING OF MODEL: MOBILE-BERT-BiRNN COMPLETED****************


------------------ PROCESS FOR MODEL: MOBILE-BERT-BiRNN COMPLETED ------------------

